In [169]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [177]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import dateutil.parser
import time
import csv
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import io
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re 
from datetime import datetime, timedelta


In [179]:
chrome_driver_path = "/usr/local/bin/chromedriver"

service = Service(chrome_driver_path)

driver = webdriver.Chrome(service=service)

driver.get("https://www.reuters.com/technology/")

In [204]:
count = 0
headlines = []
dates = []
seen_titles = set()

for x in range(10):
    try:
        driver.execute_script("document.querySelector('.ad-slot__container__FEnoz').style.display = 'none';")

        loadMoreButton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='TextButton']//span/span[text()='Load more articles']"))
        )
        loadMoreButton.click()
                
        time.sleep(3) 

        news_headlines = driver.find_elements(By.XPATH, "//h3[@data-testid='Heading']//a[@data-testid='Link' and not(span)]")
        news_dates = driver.find_elements(By.XPATH, "//time[@data-testid='Body']")
        
        for headline, date in zip(news_headlines, news_dates):
            title = headline.text
            pub_date = date.get_attribute("datetime")
            
            if title not in seen_titles:
                seen_titles.add(title)  
                headlines.append(title)
                dates.append(pub_date)
                print("Headline:", title)
                print("Date:", pub_date)
                count += 1
                print("CLICKED!!:", count)
    
    except Exception as e:
        print("Error:", e)
        break

data = pd.DataFrame({
    "Title": headlines,
    "Date": dates,
    "Category": ["Technology"] * len(headlines),
    "Source": "BBC"* len(headlines)
})

Headline: A year on, Intel's touted AI-chip deals have fallen short
Date: 2024-11-01T22:36:35Z
CLICKED!!: 1
Headline: Nvidia to take Intel's spot on Dow Jones Industrial Average
Date: 2024-11-01T17:07:46Z
CLICKED!!: 2
Headline: Crypto ETFs see big inflows ahead of U.S. election, traders brace for volatility
Date: 2024-11-01T20:10:02Z
CLICKED!!: 3
Headline: Self-proclaimed bitcoin inventor faces contempt of court case over $1.2 bln UK lawsuit
Date: 2024-11-01T16:03:50Z
CLICKED!!: 4
Headline: Apple to invest up to $1.5 bln in Globalstar for satellite coverage expansion
Date: 2024-11-01T18:17:02Z
CLICKED!!: 5
Headline: Exclusive: Walt Disney forms business unit to coordinate use of AI, augmented reality
Date: 2024-11-01T14:59:03Z
CLICKED!!: 6
Headline: Siemens can make more acquisitions after Altair deal, exec tells paper
Date: 2024-11-01T14:16:20Z
CLICKED!!: 7
Headline: France steps up efforts at examining how to protect struggling IT company Atos
Date: 2024-11-01T19:50:49Z
CLICKED!!: 8


In [206]:
data.head()

,Title,Date,Category
0,"A year on, Intel's touted AI-chip deals have f...",2024-11-01T22:36:35Z,Technology
1,Nvidia to take Intel's spot on Dow Jones Indus...,2024-11-01T17:07:46Z,Technology
2,Crypto ETFs see big inflows ahead of U.S. elec...,2024-11-01T20:10:02Z,Technology
3,Self-proclaimed bitcoin inventor faces contemp...,2024-11-01T16:03:50Z,Technology
4,Apple to invest up to $1.5 bln in Globalstar f...,2024-11-01T18:17:02Z,Technology


In [202]:
# news_headlines =driver.find_elements(By.XPATH, "//time[@data-testid='Body']")
# for headline in news_headlines:
#     headlines.append(headline.text)  

# headlines_text = [headline.text for headline in news_headlines]

# df = pd.DataFrame(headlines_text, columns=["Headline"])
# df["Category"] = "Technology"

# print(df)

             Headline    Category
0    November 2, 2024  Technology
1    November 2, 2024  Technology
2    November 2, 2024  Technology
3    November 2, 2024  Technology
4    November 2, 2024  Technology
..                ...         ...
230  October 24, 2024  Technology
231  October 24, 2024  Technology
232  October 24, 2024  Technology
233  October 24, 2024  Technology
234  October 24, 2024  Technology

[235 rows x 2 columns]


In [158]:
#BBC

In [214]:
chrome_driver_path = "/usr/local/bin/chromedriver"

service = Service(chrome_driver_path)

driver = webdriver.Chrome(service=service)
driver.get("https://www.bbc.com/innovation")


In [215]:
def convert_relative_date(relative_date_str):
    if "days ago" in relative_date_str:
        days = int(re.search(r"(\d+) days ago", relative_date_str).group(1))
        return (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
    elif "hours ago" in relative_date_str:
        hours = int(re.search(r"(\d+) hrs ago", relative_date_str).group(1))
        return (datetime.now() - timedelta(hours=hours)).strftime("%Y-%m-%d %H:%M")
    elif "yesterday" in relative_date_str:
        return (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
    else:
        return relative_date_str

all_data = []
for x in range(10):
    try:
        news_headlines = driver.find_elements(By.XPATH, "//h2[@data-testid='card-headline']")
        news_dates = driver.find_elements(By.XPATH, "//span[@data-testid='card-metadata-lastupdated']")

        for headline, date in zip(news_headlines, news_dates):
            title = headline.text
            date_text = date.text
            converted_date = convert_relative_date(date_text)
            all_data.append({"Title": title, "Date": converted_date, "Category": "Technology", "Source": "BBC"})
            print(f"Headline: {title}")
            print(f"Date: {converted_date}")

        loadMoreButton = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='pagination-next-button']"))
        )
        loadMoreButton.click()
        
        time.sleep(3)

    except Exception as e:
        print("Error:", e)
        break

data = pd.DataFrame(all_data)
data.to_csv("data.csv", index=False)

Headline: How DNA testing firm 23andMe went downhill
Date: 11 hrs ago
Headline: How children respond to daylight savings time shifts
Date: 1 day ago
Headline: 
Date: 2024-11-01
Headline: An eco-friendly plant-based alternative to plastic
Date: 2024-11-01
Headline: Snapchat most-used app for grooming, says UK charity
Date: 
Headline: 
Date: 2024-11-01
Headline: How Google tells you what you want to hear
Date: 1 day ago
Headline: Swapped at birth: How a DNA test changed two women's lives
Date: 2024-11-01
Headline: Monkeys will never type Shakespeare, study finds
Date: 26 Sep 2024
Headline: Tech Decoded: Sign up to our newsletter
Date: 12 Sep 2024
Headline: The vegetable that can protect you from cancer
Date: 29 Aug 2024
Headline: A simple and tasty way to reduce wrinkles and keep your skin healthy
Date: 15 Aug 2024
Headline: Why improving your nutrition will help you think better
Date: 25 Jul 2024
Headline: The surprising benefits of self-soothing touch
Date: 12 Jul 2024
Headline: How yo

In [218]:
data.head()

,Title,Date,Category
0,How DNA testing firm 23andMe went downhill,11 hrs ago,Technology
1,How children respond to daylight savings time ...,1 day ago,Technology
2,,2024-11-01,Technology
3,An eco-friendly plant-based alternative to pla...,2024-11-01,Technology
4,"Snapchat most-used app for grooming, says UK c...",,Technology


In [ ]:
headlines_and_dates = [] 

for headline, date in zip(news_headlines, news_dates):
    headlines_and_dates.append((headline.text, date.text))

for i, (headline, date) in enumerate(headlines_and_dates, start=1):
    print(f"Article {i}:")
    print(f"  Headline: {headline}")
    print(f"  Date: {date}")


In [ ]:
#https://medium.com/analytics-vidhya/how-to-scrape-news-headlines-from-reuters-27c0274dc13c